In [1]:
import anndata as ad
import pandas as pd
import pybedtools
import genomepy
import os 
import numpy as np


/home/jnourisa/miniconda3/envs/gimme/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# - get the gene locations as bed file

rna = ad.read_h5ad('resources/inference_datasets/op_rna.h5ad')

var = rna.var.reset_index()
var = var.iloc[:, :]
var = var[var.interval!='NA']
var[['chr','start','end']] = var.interval.str.split(':|-', expand=True)
var = var[~var.chr.isin(['GL000009.2', 'GL000194.1', 'GL000195.1', 'GL000218.1', 'GL000219.1', 'KI270711.1',
       'KI270721.1', 'KI270734.1'])]

var[['chr','start','start', 'location']].to_csv('output/skeleton/gene_locations.bed', index=False, header=False, sep='\t')

In [12]:
# - get tss flank 

# Path to the genome FASTA file
genome_fasta = os.path.expanduser('~/.local/share/genomes/hg38/hg38.fa')

# Example TSS BED file
tss_bed_file = 'output/skeleton/gene_locations.bed'

# Load the TSS BED file
tss_bed = pybedtools.BedTool(tss_bed_file)

# Set the flanking size (e.g., 1000 bp upstream and downstream)
flank_size = 1000

# Get the flanking regions
tss_with_flanks = tss_bed.slop(b=flank_size, genome=genome_fasta)

# # Save the flanking regions to a new BED file
tss_with_flanks.saveas('output/skeleton/tss_with_flanks.bed')

<BedTool(output/skeleton/tss_with_flanks.bed)>

In [5]:
# !bedtools getfasta -bed test.bed  -fi  ~/.local/share/genomes/hg38/hg38.fa 

In [13]:
# !ls /home/jnourisa/miniconda3/envs/gimme/lib/python3.10/site-packages/data/motif_databases/

In [87]:
# !gimme scan output/skeleton/tss_with_flanks.bed -g hg38 -f 0.01 -N 10 -p /home/jnourisa/miniconda3/envs/gimme/lib/python3.10/site-packages/data/motif_databases/CIS-BP.pfm -t > output/skeleton/result.scan.txt


In [8]:
df_2 = pd.read_csv('/home/jnourisa/miniconda3/envs/gimme/lib/python3.10/site-packages/data/motif_databases/HOMER.motif2factors.txt',  comment='#', sep='\t')

In [10]:
# df.columns.isin(df_2.Motif)

array([False, False, False,  True, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True, False,  True,
        True,  True,  True, False,  True,  True,  True, False,  True,
       False, False, False, False,  True, False,  True,  True,  True,
       False, False, False, False,  True,  True, False,  True,  True,
        True,  True,  True,  True, False,  True,  True, False, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True, False,  True, False,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True, False, False,
       False, False,  True, False,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [4]:
df = pd.read_csv('output/skeleton/result.scan.txt', comment='#', sep='\t')
df.columns.str.split('-', expand=True).get_level_values(0)

Index(['Unnamed: 0', 'ATGACTCATC_AP', 'SCCTSAGGSCAW_AP', 'ATGCCCTGAGGC_AP',
       'NAHCAGCTGD_Ap4(bHLH)/AML',
       'AGTAAACAAAAAAGAACANA_FOXA1:AR(Forkhead,NR)/LNCAP',
       'AGAACAGNCTGTTCTT_ARE(NR)/LNCAP', 'CCAGGAACAG_AR',
       'TBGCACGCAA_Arnt:Ahr(bHLH)/MCF7',
       'NNVVCAGCTGBN_Ascl1(bHLH)/NeuralTubes',
       ...
       'GAGSCCGAGC_ZNF519(Zf)/HEK293', 'AGAAATGACTTCCCT_ZNF528(Zf)/HEK293',
       'TTAACCCTTTVNKKN_ZNF652/HepG2', 'GARTGGTCATCGCCC_ZNF669(Zf)/HEK293',
       'ARGAGGMCAAAATGW_ZNF675(Zf)/HEK293', 'GTGGGCCCCA_ZNF692(Zf)/HEK293',
       'AGGCCTAG_ZNF711(Zf)/SHSY5Y', 'RHHCAGAGAGGB_ZNF768(Zf)/Rajj',
       'CTGCCWVCTTTTRTA_ZNF7(Zf)/HepG2',
       'SMCAGTCWGAKGGAGGAGGC_ZSCAN22(Zf)/HEK293'],
      dtype='object', length=437)

In [29]:
flank_motif  = pd.read_csv('output/skeleton/result.scan.bed', sep="\t", comment='#', names=['chr','start','end','motif','whatever','strand'], header=None)

In [79]:
# var = var.set_index(['chr' ,'start','end'])

tss_with_flanks = pd.read_csv('output/skeleton/tss_with_flanks.bed', sep="\t", comment='#', names=['chr','start','end','gene_name'], header=None)
# tss_with_flanks = tss_with_flanks.set_index(['chr' ,'start','end'])


In [75]:
tss_with_flanks

gene_name
chr  start   end                
chr1 119931  121931   AL627309.1
     148706  150706   AL627309.5
     159445  161445   AL627309.4
     777757  779757    LINC01409
     826597  828597    LINC01128
...                          ...
     5667254 5669254  AL365255.1
     5991424 5993424       NPHP4
     5991261 5993261      KCNAB2
     6179320 6181320        CHD5
     6198594 6200594       RPL22

[100 rows x 1 columns]

In [80]:
flank_tf.start.isin(tss_with_flanks.start)

2        False
3        False
4        False
5        False
8        False
         ...  
47784    False
47785    False
47786    False
47787    False
47788    False
Name: start, Length: 33001, dtype: bool

In [77]:
# - make flank_tf out of flank_motif 
motif2tf = pd.read_csv('/home/jnourisa/miniconda3/envs/gimme/lib/python3.10/site-packages/data/motif_databases/CIS-BP.motif2factors.txt', sep='\t') 
motif2tf.columns = ['motif','tf','_','__']

flank_tf = flank_motif.merge(motif2tf, on='motif', how='left')
flank_tf = flank_tf[~flank_tf.tf.isna()]
# flank_tf = flank_tf.set_index(['chr' ,'start','end'])
# var.merge(tss_with_flanks, left_index=True, right_index=True)

In [53]:
tf_list = np.loadtxt('resources/prior/tf_all.csv', dtype=str)

In [25]:
import pandas as pd

column_names = ['chrom', 'start', 'end', 'name', 'score', 'strand']

# Read the BED file with appropriate settings
df = pd.read_csv(
    'output/skeleton/result.scan.bed',
    sep='\t',
    comment='#',      # Skips comment lines starting with '#'
    header=None,      # Indicates there is no header in the file
    names=column_names # Assigns names to columns
)

df

AttributeError: 'MultiIndex' object has no attribute 'split'